In [ ]:
# Import Required Libraries
import cv2
import numpy as np


In [ ]:
# Function to Draw Lines
def draw_lines(frame, lines):
    line_image = np.zeros_like(frame)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 5)
    return line_image


In [ ]:
# Region of Interest (ROI)
def region_of_interest(edges):
    height, width = edges.shape
    mask = np.zeros_like(edges)

    # Triangle covering bottom half of the frame (road area)
    polygon = np.array([[
        (0, height),
        (width, height),
        (int(width / 2), int(height / 2) + 50)
    ]], np.int32)

    cv2.fillPoly(mask, polygon, 255)
    cropped_edges = cv2.bitwise_and(edges, mask)
    return cropped_edges


In [ ]:
# Process Each Video Frame
def process_frame(frame):
    # 1. Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 2. Apply Gaussian blur to reduce noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # 3. Detect edges using Canny
    edges = cv2.Canny(blur, 50, 150)

    # 4. Apply region of interest
    roi = region_of_interest(edges)

    # 5. Detect straight lines using Hough Transform
    lines = cv2.HoughLinesP(
        roi,
        rho=2,                    # distance resolution in pixels
        theta=np.pi/180,          # angle resolution in radians
        threshold=100,            # minimum number of votes (strength of line)
        minLineLength=40,         # minimum length of a line
        maxLineGap=5              # maximum gap between lines to connect
    )

    # 6. Draw the lines
    line_image = draw_lines(frame, lines)

    # 7. Combine original frame with detected lines
    combo = cv2.addWeighted(frame, 0.8, line_image, 1, 1)

    return combo


In [9]:
# Load and Process Video
video_path = 'highway.mp4'  # Your video file
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

print("Press 's' to exit the video window.")


Press 's' to exit the video window.


In [15]:
# Read and Display Frames
while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video reached or cannot read frame.")
        break

    # Optional: resize for faster processing
    frame = cv2.resize(frame, (960, 540))

    # Process the frame
    lane_frame = process_frame(frame)

    # Show the video
    cv2.imshow("Lane Detection - Highway", lane_frame)

    # Press 'q' to stop
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


End of video reached or cannot read frame.
